In [1]:
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

2.4.3
2.4.1


In [2]:
from __future__ import print_function

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
EPOCHS = 5
YEAR = '2000'
#YEAR = '2010'

df = pd.read_csv('../../data/census/census_%s.csv' % YEAR)
df.dropna(subset=['name'], inplace=True)
df.replace('(S)', 0, inplace=True)
df

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,SMITH,1,2376206,880.85,880.85,73.35,22.22,0.40,0.85,1.63,1.56
1,JOHNSON,2,1857160,688.44,1569.30,61.55,33.80,0.42,0.91,1.82,1.50
2,WILLIAMS,3,1534042,568.66,2137.96,48.52,46.72,0.37,0.78,2.01,1.60
3,BROWN,4,1380145,511.62,2649.58,60.71,34.54,0.41,0.83,1.86,1.64
4,JONES,5,1362755,505.17,3154.75,57.69,37.73,0.35,0.94,1.85,1.44
...,...,...,...,...,...,...,...,...,...,...,...
151666,ZILK,150436,100,0.04,89753.41,90.00,9.00,0.00,0.00,0,0
151667,ZINNANTI,150436,100,0.04,89753.45,98.00,0.00,0.00,0.00,0,0
151668,ZITTERICH,150436,100,0.04,89753.48,98.00,0,0.00,0.00,0.00,0
151669,ZULU,150436,100,0.04,89753.52,6.00,90.00,0.00,0.00,0,0


## Resampling with weight

In [3]:
sdf = df.sample(1000000, weights=df['count'], replace=True)

In [4]:
sdf

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
44404,OYEN,44350,457,0.17,81723.65,94.75,0,2.19,1.09,0,0
16573,GUINAN,16571,1595,0.59,73337.72,93.23,0,1.44,0,1.07,3.45
16,MARTIN,17,672711,249.37,6747.54,77.47,15.30,0.71,0.94,1.59,3.99
6877,RAYMER,6877,4507,1.67,63850.28,93.90,1.82,0.42,0.53,1.53,1.80
4197,MAESTAS,4198,7830,2.90,57996.14,14.96,0.24,0.26,1.92,0.92,81.71
...,...,...,...,...,...,...,...,...,...,...,...
7457,MORTENSON,7457,4116,1.53,64774.91,94.95,0.27,0.66,1.34,1.09,1.70
6585,MELCHER,6585,4745,1.76,63350.01,94.75,0.51,0.93,0.36,1.31,2.15
13333,HENRICKSON,13333,2097,0.78,71144.53,94.99,0.38,0.67,0.33,1.38,2.24
607,VINCENT,608,50628,18.77,34778.65,82.26,12.05,0.94,0.60,1.83,2.33


## Assign race by pertcentage

In [5]:
from numpy.random import choice

races = ['white', 'black', 'api', 'hispanic']

def to_race(c):
    w = np.array(c).astype(float)
    if w.sum() == 0:
        return 'white'
    probs = w/w.sum()    
    return choice(races, p=probs)

sdf['race'] = sdf[['pctwhite', 'pctblack', 'pctapi', 'pcthispanic']].apply(lambda c: to_race(c), axis=1)


## Check the correctness of race assignment

In [6]:
df[df.name == 'SMITH']

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,SMITH,1,2376206,880.85,880.85,73.35,22.22,0.40,0.85,1.63,1.56


In [7]:
xdf = sdf[sdf.name=='SMITH'].groupby(['race']).agg({'name': 'count'})
xdf * 100 / xdf.sum()

,name
race,
api,0.451143
black,23.274890
hispanic,1.466216
white,74.807751


In [8]:
# Additional features
sdf['name_last'] = sdf.name.str.title()
sdf.groupby('race').agg({'name_last': 'count'})

,name_last
race,
api,35256
black,125993
hispanic,128261
white,710490


In [9]:
len(sdf)

1000000

## Preprocessing the input data

In [10]:
# only last name in Census data
sdf['name_last_name_first'] = sdf['name_last']

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
#vect = CountVectorizer(analyzer='char', ngram_range=(2, 2), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_
len(vocab)

961

In [11]:
import operator
sorted_vocab = sorted(vocab.items(), key=operator.itemgetter(1))
cols = list(map(operator.itemgetter(0), sorted_vocab))

In [12]:
count_df = pd.DataFrame(a.todense(), columns=cols)
count_df

,Aa,Ab,Ac,Ad,Ae,Af,Ag,Ah,Ai,Aj,...,zp,zq,zr,zs,zt,zu,zv,zw,zy,zz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
999996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
999997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
999998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
count_df.sum().sort_values(ascending=False).describe()

count       961.000000
mean       5520.249740
std       12675.471668
min           3.000000
25%          80.000000
50%         917.000000
75%        5416.000000
max      175925.000000
dtype: float64

In [14]:
pd.set_option('display.max_rows', 20)
count_df.sum().sort_values(ascending=False)

er    175925
on    124027
an    106923
ar    100093
ll     86582
       ...  
zj         3
pz         3
qe         3
Mh         3
Ks         3
Length: 961, dtype: int64

In [15]:
# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = [w[1] for w in words]
num_words = len(words_list)
print("num_words = %d" % num_words)


def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

num_words = 961
Max feature len = 14, Avg. feature len = 5


In [16]:
len(vocab)

961

## Train a LSTM model

ref: http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [17]:
'''The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model

max_features = num_words # 20000
feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

800000 train sequences
200000 test sequences
Pad sequences (samples x time)
X_train shape: (800000, 20)
X_test shape: (200000, 20)
4 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (800000, 4)
y_test shape: (200000, 4)


In [18]:
print('Build model...')

model = Sequential()
model.add(Embedding(num_words, 32, input_length=feature_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 32)            30752     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               82432     
_________________________________________________________________
dense (Dense)                (None, 4)                 516       
Total params: 113,700
Trainable params: 113,700
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=2)
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/5
22500/22500 - 527s - loss: 0.5671 - accuracy: 0.8057 - val_loss: 0.5372 - val_accuracy: 0.8154
Epoch 2/5
22500/22500 - 921s - loss: 0.5296 - accuracy: 0.8174 - val_loss: 0.5210 - val_accuracy: 0.8195
Epoch 3/5
22500/22500 - 1114s - loss: 0.5162 - accuracy: 0.8209 - val_loss: 0.5128 - val_accuracy: 0.8219
Epoch 4/5
22500/22500 - 1070s - loss: 0.5087 - accuracy: 0.8229 - val_loss: 0.5062 - val_accuracy: 0.8235
Epoch 5/5
22500/22500 - 786s - loss: 0.5041 - accuracy: 0.8240 - val_loss: 0.5051 - val_accuracy: 0.8243
6250/6250 - 31s - loss: 0.4999 - accuracy: 0.8253
Test score: 0.4998873770236969
Test accuracy: 0.8253499865531921


```
Train...
Train on 72000 samples, validate on 8000 samples
Epoch 1/5
124s - loss: 0.6793 - acc: 0.7678 - val_loss: 0.6147 - val_acc: 0.7917
Epoch 2/5
156s - loss: 0.5908 - acc: 0.7979 - val_loss: 0.5909 - val_acc: 0.8001
Epoch 3/5
157s - loss: 0.5725 - acc: 0.8043 - val_loss: 0.5792 - val_acc: 0.8031
Epoch 4/5
206s - loss: 0.5621 - acc: 0.8071 - val_loss: 0.5726 - val_acc: 0.8039
Epoch 5/5
156s - loss: 0.5556 - acc: 0.8087 - val_loss: 0.5741 - val_acc: 0.8044
Test score: 0.56178010149
Test accuracy: 0.80655
```

## Confusion Matrix

In [20]:
y_pred = model.predict_classes(X_test, verbose=2)
p = model.predict_proba(X_test, verbose=2) # to predict probability
target_names = list(sdf.race.astype('category').cat.categories)
print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

/Users/madhusanjeevi/Desktop/GauravFreelanceing/env/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


6250/6250 - 29s


/Users/madhusanjeevi/Desktop/GauravFreelanceing/env/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


6250/6250 - 28s
              precision    recall  f1-score   support

         api       0.88      0.62      0.72      7051
       black       0.51      0.07      0.12     25199
    hispanic       0.86      0.84      0.85     25652
       white       0.83      0.97      0.89    142098

    accuracy                           0.83    200000
   macro avg       0.77      0.62      0.65    200000
weighted avg       0.79      0.83      0.78    200000

[[  4343     22    670   2016]
 [    88   1788    206  23117]
 [    98     62  21572   3920]
 [   403   1638   2690 137367]]


## Save model

In [21]:
model.save('./models/census/lstm/census%s_ln_lstm.h5' % YEAR)

In [22]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('./models/census/lstm/census%s_ln_vocab.csv' % YEAR, index=False, encoding='utf-8')